In [10]:
import pandas as pd
import pdfplumber
import re
import json
import os

In [2]:
path_who = r"C:\Users\Dee\root\Projects\dev\unpatternedAi\unstructureparser\docs\world_health_statistics_2024.pdf"

In [3]:
with pdfplumber.open(path_who) as pdf:
    page = pdf.pages[2]
    text = page.extract_text()
    
    for line in text.split('\n'):
        print(line)

World health
statistics 2024
Monitoring health for the SDGs,
Sustainable Development Goals


In [4]:
print(text)

World health
statistics 2024
Monitoring health for the SDGs,
Sustainable Development Goals


In [5]:
def extract_text_from_pdf(pdf_path):
    """ Extracts text from a PDF file and returns a list of dicitionaries with page numbers and text"""
    extracted_data = []
    
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"The file '{pdf_path}' does not exist.")
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_num = page.page_number
            print(f"... Extracting text information from page {page_num} ...")
            str_text = page.extract_text()
            if str_text: # Only store non-empth pages
                extracted_data.append({
                    "page": page_num,
                    "text": str_text,
                    "source": os.path.basename(pdf_path)
                })
    return extracted_data

In [6]:
def save_extracted_data(data, output_path):
    """Saves extracted text data to a JSON file."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"Extracted data saved to {output_path}")

In [7]:
def process_pdf(pdf_path, output_path):
    """Main function to process the PDF and save extracted text."""
    try:
        extracted_data = extract_text_from_pdf(pdf_path)
        save_extracted_data(extracted_data, output_path)
    except Exception as e:
        print(f"Error processing PDF: {e}")

In [8]:
who_pdf_path = r"C:\Users\Dee\root\Projects\dev\unpatternedAi\unstructureparser\docs\world_health_statistics_2024.pdf"
output_who_pdf_path = r"C:\Users\Dee\root\Projects\dev\unpatternedAi\unstructureparser\data\raw\who_extracted_pdf.json"

In [11]:
process_pdf(who_pdf_path, output_who_pdf_path)

... Extracting text information from page 1 ...
... Extracting text information from page 2 ...
... Extracting text information from page 3 ...
... Extracting text information from page 4 ...
... Extracting text information from page 5 ...
... Extracting text information from page 6 ...
... Extracting text information from page 7 ...
... Extracting text information from page 8 ...
... Extracting text information from page 9 ...
... Extracting text information from page 10 ...
... Extracting text information from page 11 ...
... Extracting text information from page 12 ...
... Extracting text information from page 13 ...
... Extracting text information from page 14 ...
... Extracting text information from page 15 ...
... Extracting text information from page 16 ...
... Extracting text information from page 17 ...
... Extracting text information from page 18 ...
... Extracting text information from page 19 ...
... Extracting text information from page 20 ...
... Extracting text informati

In [12]:
from dotenv import  load_dotenv
load_dotenv()

True

In [14]:
raw_json = r"C:\Users\Dee\root\Projects\dev\unpatternedAi\unstructureparser\data\raw\who_extracted_pdf.json"

In [19]:
openai_client = (api_key=os.getenv("OPENAI_API_KEY", "your-openai-api-key"))

NameError: name 'OpenAI' is not defined

In [21]:
import os
import json
from openai import OpenAI
# 1. Modern OpenAI Client Initialization
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "your-openai-api-key"))

In [25]:
# 7. Function to generate an answer using OpenAI's updated API call.
def generate_answer(query):
    """
    Combine the query and the retrieved context to generate an answer using OpenAI's Chat API.
    """
   #context = "\n\n".join(retrieved_context)
    prompt = (
        f"You are an expert on global health reports. Given the following context extracted from WHO reports:\n"
        f"Answer the following question in a detailed and helpful manner:\n"
        f"{query}\n"
    )
    
    try:
        # Updated API call using client.chat.completions.create
        # Updated API call using client.chat.completions.create
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",  # or "gpt-4" if available
            messages=[
                {"role": "system", "content": "You are an assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=200
        )
        
        # Parse and return the message content as JSON.
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Error generating answer: {e}")
        return {"error": "Sorry, I couldn't generate an answer."}

In [ ]:
query = "What are the global trends in maternal health?"


In [26]:
generate_answer(query)

Error generating answer: Expecting value: line 1 column 1 (char 0)


{'error': "Sorry, I couldn't generate an answer."}

In [30]:
import os
import json
from openai import OpenAI
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# 1. Configure your OpenAI API key.
openai_api_key = os.getenv("OPENAI_API_KEY") or "your-openai-api-key"
openai.api_key = openai_api_key

# 1. Modern OpenAI Client Initialization
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "your-openai-api-key"))

# 2. Create an embedding function for ChromaDB using OpenAI.
embedding_function = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name="text-embedding-ada-002")

# 3. Connect (or create) a persistent ChromaDB client.
client = chromadb.PersistentClient(path="./vector_db", settings=Settings())
collection_name = "who_publications"
collection = client.get_or_create_collection(
    name=collection_name, 
    embedding_function=embedding_function
)

# 4. Function to load the extracted JSON file.
def load_extracted_json(json_path):
    """Loads the JSON file containing extracted PDF text data."""
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"JSON file not found: {json_path}")
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)

# 5. Function to store documents in ChromaDB.
def store_documents(documents):
    """
    Given a list of document dicts (with keys 'source', 'page', and 'text'),
    store them in ChromaDB, including both the text and embeddings.
    """
    for item in documents:
        doc_id = f"{item['source']}_page_{item['page']}"
        print(f"-> Loading document: {doc_id}")
        collection.add(
            ids=[doc_id],
            documents=[item["text"]],  # Ensure actual text is stored!
            metadatas=[{"page": item["page"], "source": item["source"]}],
        )
        print(f"Document {doc_id} successfully stored in ChromaDB.")

# 6. Function to retrieve similar documents from ChromaDB.
def retrieve_similar_documents(query, top_k=3):
    """
    Retrieve similar documents based on the query from ChromaDB.
    """
    results = collection.query(query_texts=[query], n_results=top_k)
    
    # Verify that we have valid document text in the results.
    if not results or "documents" not in results or not results["documents"]:
        return []
    
    matched_texts = results["documents"][0] if results["documents"][0] else []
    return [str(doc) for doc in matched_texts]  # Ensure all docs are strings

# 7. Function to generate an answer using OpenAI's updated API call.
def generate_answer(query, retrieved_context):
    """
    Combine the query and the retrieved context to generate an answer using OpenAI's Chat API.
    Focus on a Monitoring, Learning, and Evaluation (MLE) perspective.
    The response should explain if it remains a global challenge,
    highlighting mortality rates, economic implications, and persistent challenges.
    """
    context = "\n\n".join(retrieved_context)
    prompt = (
        "You are an expert in global health, impact, and evaluation, specializing in Monitoring, "
        "Learning, and Evaluation (MLE). Given the following context extracted from WHO reports:\n"
        f"{context}\n\n"
        "Answer the following question in a detailed and analytical manner. "
        "Add number and statistics from WHO publication documents"
        "Your answer should assess whether the issue remains a critical global problem, "
        "focusing on mortality rates, economic costs, demographic, and ongoing challenges related to that health issue. "
        f"Question: {query}\n"
    )
    
    try:
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",  # or "gpt-4" if available
            messages=[
                {"role": "system", "content": "You are an assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        # Return the plain text answer.
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating answer: {e}")
        return {"error": "Sorry, I couldn't generate an answer."}

# 8. Main workflow.
if __name__ == "__main__":
    try:
        # Define the JSON file path (adjust as needed).
        # json_path = os.path.join(os.getcwd(), "data", "raw", "who_extracted_pdf.json")
        # 
        # # Load the extracted data from JSON.
        # parsed_json = load_extracted_json(json_path)
        # print(f"Loaded {len(parsed_json)} documents from JSON.")
        
        # (Optional) Delete existing records from ChromaDB before re-loading.
        # collection.delete()  # Uncomment if you wish to remove existing documents.
        # 
        # # Store documents into ChromaDB.
        # store_documents(parsed_json)
        
        # Use a sample query.
        query = "What are the global trends in maternal health?"
        retrieved_docs = retrieve_similar_documents(query, top_k=5)
        
        # Do not display retrieved context to the user; use it directly to generate the answer.
        final_answer = generate_answer(query, retrieved_docs)
        print("\nFinal Answer Generated by OpenAI:\n")
        print(final_answer)
    except Exception as e:
        print(f"Error: {e}")



Final Answer Generated by OpenAI:

Maternal health remains a critical global issue despite some progress made in reducing maternal mortality rates (MMR) over the years. According to WHO reports, the global MMR dropped significantly during the Millennium Development Goals (MDG) era, from 339 deaths per 100,000 live births in 2000 to 227 deaths per 100,000 live births in 2015. However, progress has stagnated since then, with the global MMR only slightly decreasing to 223 deaths per 100,000 live births in 2020.

The global MMR reduction has not been consistent across all regions. The African Region has consistently had the highest MMR, despite a 2% average annual rate of reduction. On the other hand, the South-East Asia Region experienced the steepest decline in MMR. This disparity in progress highlights the ongoing challenges and disparities in maternal health outcomes globally.

To achieve the Sustainable Development Goals (SDG) global target of an MMR below 70 deaths per 100,000 live 

In [16]:
query = "What are the global trends in maternal health?"
retrieved_docs = retrieve_similar_documents(query, top_k=3)

# Display retrieved context.
display_retrieved_context(query, retrieved_docs)

# Generate a final answer using OpenAI based on retrieved documents.
final_answer = generate_answer(query, retrieved_docs)
print("\nFinal Answer Generated by OpenAI:\n")
print(final_answer)


Top 3 Document Excerpts for Query: 'What are the global trends in maternal health?':

1. 4
Key issues and trends
in global health
The global population and economic growth, the
advancement of science and technology, and the ever-
increasing interconnectedness of the world have brought
about enormous societal changes along with unprecedented
progress, but also multiple and complex challe...

2. Figure 2.11 Percentage change in adolescent birth rate, by age group, and the proportion of births among
adolescent girls out of total births, globally and by WHO region, 2015–2023
Eastern Region of the South-East Asia Western Pacific
Mediterranean Americas African Region Region Global Region Europe...

3. 3.
Progress towards WHO Triple Billion targets 55
3.1 Healthier populations billion 56
3.2 UHC billion 58
3.3 Health emergencies protection billion 60
3.4 Health-related SDGs and health information system 61
3.5 Conclusion 64
4.
Key issues and trends in global health 65
4.1 The double burden o

In [ ]:
import os
import json
import openai
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# 1. Configure your OpenAI API key.
openai_api_key = os.getenv("OPENAI_API_KEY") or "your-openai-api-key"
openai.api_key = openai_api_key

# 2. Create an embedding function for ChromaDB using OpenAI.
embedding_function = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name="text-embedding-ada-002")

# 3. Connect (or create) a persistent ChromaDB client.
client = chromadb.PersistentClient(path="./vector_db", settings=Settings())
collection_name = "who_publications"
collection = client.get_or_create_collection(
    name=collection_name, 
    embedding_function=embedding_function
)

# 4. Function to load (or re-load) documents into ChromaDB.
def store_documents(parsed_json):
    """
    Given a list of document dicts (with keys 'source', 'page', and 'text'),
    store them in ChromaDB, including both text and embeddings.
    """
    for item in parsed_json:
        doc_id = f"{item['source']}_page_{item['page']}"
        print(f"-> Loading document: {doc_id}")
        # When using OpenAI embedding function with ChromaDB, you don't need to pre-compute embeddings.
        collection.add(
            ids=[doc_id],
            documents=[item["text"]],  # Ensure the actual text is stored.
            metadatas=[{"page": item["page"], "source": item["source"]}],
        )
        print(f"Document {doc_id} successfully stored in ChromaDB.")

# 5. Function to retrieve similar documents from ChromaDB.
def retrieve_similar_documents(query, top_k=3):
    """
    Retrieve similar documents based on the query from ChromaDB.
    """
    results = collection.query(query_texts=[query], n_results=top_k)
    
    # Verify that we have valid document text in the results.
    if not results or "documents" not in results or not results["documents"]:
        return []
    
    matched_texts = results["documents"][0] if results["documents"][0] else []
    return [str(doc) for doc in matched_texts]  # Convert to strings if needed

# 6. Function to generate an answer using OpenAI's updated API call.
def generate_answer(query, retrieved_context):
    """
    Combine the query and the retrieved context to generate an answer using OpenAI's Chat API.
    """
    context = "\n\n".join(retrieved_context)
    prompt = (
        f"You are an expert on global health reports. Given the following context extracted from WHO reports:\n"
        f"{context}\n\n"
        f"Answer the following question in a detailed and helpful manner:\n"
        f"{query}\n"
    )
    #     response = openai_client.chat.completions.create(
    #     model="gpt-4-turbo-preview",  # Use latest model
    #     response_format={"type": "json_object"},  # Enforce JSON output
    #     messages=[{
    #         "role": "system",
    #         "content": """You are a WHO health analyst. Answer questions using provided context. 
    #                     Include citations using [Source: filename, Page X]. Format response as JSON with 
    #                     'answer', 'confidence' (0-1), and 'citations' fields."""
    #     }, {
    #         "role": "user",
    #         "content": f"Context:\n{context_str}\n\nQuestion: {query}"
    #     }],
    #     temperature=0.2,
    #     max_tokens=500
    # )
    # 
    try:
        # Updated API call using client.chat.completions.create
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",  # or "gpt-4" if available
            messages=[
                {"role": "system", "content": "You are an assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=200
        )
        
    
        # Parse and return the message content as JSON.
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Error generating answer: {e}")
        return {"error": "Sorry, I couldn't generate an answer."}

# 7. Optionally, a helper function to display the retrieved context.
def display_retrieved_context(query, retrieved_docs):
    if not retrieved_docs:
        print(f"\nNo matching results found for query: '{query}'\n")
    else:
        print(f"\nTop {len(retrieved_docs)} Document Excerpts for Query: '{query}':\n")
        for idx, doc in enumerate(retrieved_docs):
            print(f"{idx+1}. {doc[:300]}...\n")  # Show first 300 characters per document

# 8. Example workflow.
if __name__ == "__main__":
    # If you haven't already loaded your documents:
    # parsed_json should be a list of dictionaries with keys: 'source', 'page', 'text'.
    # For example:
    # parsed_json = [
    #    {"source": "world_health_statistics_2024.pdf", "page": 1, "text": "Your document text here..."},
    #    {"source": "world_health_statistics_2024.pdf", "page": 2, "text": "More text..."},
    #    ... 
    # ]
    # Uncomment below to store documents:
    # store_documents(parsed_json)

    # Use a sample query.
    query = "What are the global trends in maternal health?"
    retrieved_docs = retrieve_similar_documents(query, top_k=3)
    
    # Display the retrieved context (optional).
    display_retrieved_context(query, retrieved_docs)
    
    # Generate a final answer using OpenAI based on retrieved documents.
    final_answer = generate_answer(query, retrieved_docs)
    print("\nFinal Answer Generated by OpenAI:\n")
    print(final_answer)


In [ ]:
import os
import json
from openai import OpenAI
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from tenacity import retry, stop_after_attempt, wait_exponential
from typing import List, Dict, Any

# 1. Modern OpenAI Client Initialization
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "your-openai-api-key"))

# 2. Enhanced ChromaDB Configuration
class VectorDBManager:
    def __init__(self, path: str = "./vector_db"):
        self.client = chromadb.PersistentClient(
            path=path,
            settings=Settings(allow_reset=True, anonymized_telemetry=False)
        )
        self.embedding_function = OpenAIEmbeddingFunction(
            api_key=openai_client.api_key,
            model_name="text-embedding-3-small"  # Updated to latest embedding model
        )
    
    def get_collection(self, name: str):
        return self.client.get_or_create_collection(
            name=name,
            embedding_function=self.embedding_function,
            metadata={"hnsw:space": "cosine"}  # Optimized similarity metric
        )

vector_db = VectorDBManager()
collection = vector_db.get_collection("who_publications_v2")

# 3. Batch Document Processing with Metadata
def store_documents(documents: List[Dict[str, Any]], batch_size: int = 100):
    """Batch process documents with improved metadata handling"""
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        ids = [f"{doc['source']}_pg_{doc['page']}_{i}" for i, doc in enumerate(batch)]
        texts = [doc["text"] for doc in batch]
        metadatas = [{
            "source": doc["source"],
            "page": doc["page"],
            "doc_type": "WHO_report",  # Add structured metadata
            "timestamp": doc.get("timestamp", "2023-01-01")  # Example metadata field
        } for doc in batch]
        
        collection.upsert(
            ids=ids,
            documents=texts,
            metadatas=metadatas
        )
        print(f"Processed batch {i//batch_size + 1}/{(len(documents)-1)//batch_size + 1}")

# 4. Enhanced Retrieval with Hybrid Search
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def retrieve_context(query: str, top_k: int = 5, filter: Dict = None) -> List[Dict]:
    """Hybrid search with metadata filtering"""
    results = collection.query(
        query_texts=[query],
        n_results=top_k,
        where=filter,  # Metadata filtering
        include=["documents", "metadatas", "distances"]
    )
    
    return [{
        "text": doc,
        "metadata": meta,
        "score": score
    } for doc, meta, score in zip(
        results["documents"][0],
        results["metadatas"][0],
        results["distances"][0]
    )]

# 5. Modern Chat Completion with Structured Output
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def generate_answer(query: str, context: List[Dict]) -> Dict[str, Any]:
    """Generate answer with citations using latest API"""
    context_str = "\n\n".join(
        f"Source: {c['metadata']['source']} (Page {c['metadata']['page']}):\n{c['text']}"
        for c in context
    )
    
    response = openai_client.chat.completions.create(
        model="gpt-4-turbo-preview",  # Use latest model
        response_format={"type": "json_object"},  # Enforce JSON output
        messages=[{
            "role": "system",
            "content": """You are a WHO health analyst. Answer questions using provided context. 
                        Include citations using [Source: filename, Page X]. Format response as JSON with 
                        'answer', 'confidence' (0-1), and 'citations' fields."""
        }, {
            "role": "user",
            "content": f"Context:\n{context_str}\n\nQuestion: {query}"
        }],
        temperature=0.2,
        max_tokens=500
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except json.JSONDecodeError:
        return {"error": "Failed to parse model response"}

# 6. Validation and Error Handling
def validate_document_structure(docs: List[Dict]) -> bool:
    required_keys = {"source", "page", "text"}
    return all(required_keys.issubset(doc.keys()) for doc in docs)

# 7. Example Workflow with Improvements
if __name__ == "__main__":
    # Sample document validation and ingestion
    sample_docs = [
        {"source": "world_health_2024.pdf", "page": 1, "text": "..."},
        # ... other documents
    ]
    
    if validate_document_structure(sample_docs):
        store_documents(sample_docs, batch_size=50)
    else:
        print("Document structure validation failed")
        exit(1)
    
    # Query with metadata filtering
    query = "What are the latest maternal mortality rates in Africa?"
    context = retrieve_context(
        query,
        top_k=5,
        filter={"doc_type": "WHO_report", "timestamp": {"$gte": "2022-01-01"}}
    )
    
    # Generate and display answer
    if context:
        answer = generate_answer(query, context)
        print("\nExpert Analysis:")
        print(answer.get("answer", "No answer generated"))
        print("\nCitations:")
        for cite in answer.get("citations", []):
            print(f"- {cite}")
    else:
        print("No relevant documents found")